Rewards are flat from trial 1, would the optimal adversary which introduces lower rewards be better? It's ASR is like 25% with ACG, but that might be okay for training. NB thatACG already slows training, and failing entails using the entire step/restart budget 

In [1]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

import gym
import numpy as np
from datetime import datetime

from art.attacks.evasion import AutoConjugateGradient as ACG


from KBMproject import ATLA
import KBMproject.utilities as utils

from citylearn.data import DataSet

In [2]:
DATASET_NAME = 'citylearn_challenge_2022_phase_2'
SAVE_DIR = 'Models/ACG/'
LOG_DIR = 'logs/Phase3/ACG/'
VERBOSITY = 0
DEVICE = 'cuda'

In [3]:
BINS = 20
EVALS = 60
AGENT = '20 bin PPO 500 results/default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
MAX_EPISODES = 200
EPS = 0.05

In [4]:
agent = PPO.load(path=AGENT,
                    #env=agent_env,
                    device=DEVICE,
                    verbose=VERBOSITY,
                    print_system_info=True,
                    )

== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.22631-SP0 10.0.22631
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.0
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0



In [5]:
kwargs = dict(
    schema=DataSet.get_schema(DATASET_NAME),
    action_bins=agent.action_space[0].n,
    T=None #this was supposed to make evaluations shorter, but does not work... never passed it in lol
)
env = utils.make_discrete_env(**kwargs,seed=0)
eval_env = utils.make_discrete_env(**kwargs,seed=42)

T = env.time_steps - 1

ACG chosen for:
- confidence bound, agent robust within eps
- A min norm attack like BB will increas it's budget to be successful, making it difficult for the agent to overcome

In [6]:
mask_time = np.ones(agent.observation_space.shape)
mask_time[0:6] = 0 #mask time features

In [7]:
init = 50
iter = int(500/init)
kwargs = dict(
    loss_type='difference_logits_ratio', 
    nb_random_init=init, #5, lower values speed crafting
    max_iter=iter, #iterations per restart
    norm='inf', #->l2 ->l1 most restrictive
    eps = EPS,
    eps_step = 2*EPS,
    verbose=False, #no progress bar
)
perturbation = ATLA.ARTperturbation(
                                    art_attack=utils.define_attack(agent, ACG, **kwargs),
                                    generate_kwargs=dict(
                                                        mask=mask_time
                                                        )
                                )

Assign wrapped environment to agent

In [8]:
env = ATLA.VictimATLAWrapper(env,
                                   perturbation,)
env = Monitor(env)
agent.set_env(env)

eval_env = ATLA.VictimATLAWrapper(eval_env,
                                  perturbation,)
eval_env = Monitor(eval_env)

In [9]:
check_env(env)

In [10]:
kwargs = dict(
    eval_freq=MAX_EPISODES//EVALS*T,
    verbose=VERBOSITY
)

agent_eval_callback = EvalCallback(eval_env,**kwargs)

In [11]:
agent_n_ts = agent.num_timesteps
now = datetime.now()
dtg = f'{now.month}-{now.day}-{now.hour}'

In [12]:
agent_name = f'{agent.__class__.__name__} agent '
agent_name += f'{agent_n_ts//T}+{MAX_EPISODES} ACG adv {dtg}'

error with torch 1.2.0 updated to 1.2.1

In [13]:
agent.learn(total_timesteps=MAX_EPISODES*T,
            callback=[agent_eval_callback,
                      ATLA.HParamCallback(),
                      ],
                tb_log_name=agent_name,
                log_interval=1,
                )

RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
if SAVE_DIR is not None:
    agent.save(SAVE_DIR + agent_name)